In [1]:
import os
import sys
import numpy as np
from scipy.sparse import csr_matrix
if '..' not in sys.path:
    sys.path.append('..')
from pfcommon import parse_sparse_matrix_file, parse_Amat_vars_file, parse_Jacobian_vars_file

In [2]:
def print_matrix(M, var_names, outfile=None):
    n_vars = len(var_names)
    n_char = max(map(len, var_names))
    fmt_str = '  {:>' + str(n_char) + 's} '
    fmt_num = '  {:' + str(n_char) + 'g} '
    if outfile is not None:
        out = open(outfile,'w')
    else:
        out = sys.stdout
    out.write(' ' * 5)
    for var_name in var_names:
        out.write(fmt_str.format(var_name))
    out.write('\n')
    for i,row in enumerate(M):
        out.write(f'[{i+1:02d}] ')
        for val in row:
            out.write(fmt_num.format(val))
        out.write('\n')
    if outfile is not None:
        out.close()

In [3]:
folder = os.path.join('..','..','modal_analysis','SM_with_load','default')
filename = os.path.join(folder, 'VariableToIdx_Jacobian.txt')
vars_idx,state_vars,voltages,currents,signals = parse_Jacobian_vars_file(filename)
filename = os.path.join(folder, 'Jacobian.mtl')
J = parse_sparse_matrix_file(filename, )
filename = os.path.join(folder, 'SM_with_load_AC.npz')
data = np.load(filename, allow_pickle=True)
S = data['S'].item()
PF = data['PF_without_slack'].item()
PF_bus = PF['buses']['Bus1']
PF_load = PF['loads']['LD1']
PF_gen = PF['SMs']['G1']
cosphi = PF_gen['cosphi']
ϕ = np.arccos(cosphi)
print('cos(ϕ) = {:7.3f}'.format(cosphi))
print('     ϕ = {:7.3f} deg'.format(np.rad2deg(ϕ)))

cos(ϕ) =   0.981
     ϕ =  11.310 deg


In [4]:
flatten = lambda D: [k+'.'+el for k,subl in D.items() for el in subl]
var_names = flatten(state_vars) + flatten(voltages) + flatten(currents)
n_vars = len(var_names)
print('State variables: "{}".'.format('", "'.join(flatten(state_vars))))
print('       Voltages: "{}".'.format('", "'.join(flatten(voltages))))
print('       Currents: "{}".'.format('", "'.join(flatten(currents))))

State variables: "G1.speed", "G1.phi".
       Voltages: "Bus1.ur", "Bus1.ui".
       Currents: "G1.ir:bus1", "G1.ii:bus1", "LD1.ir:bus1", "LD1.ii:bus1".


#### Base parameters
These values are used by PowerFactory for display purposes

In [5]:
F      = 50.  # [Hz] default frequency
S_base = 1e6  # [VA] base apparent power
V_base = PF_bus['Vl']/PF_bus['u']*1e3  # [V] base voltage (line-to-line)
I_base = S_base/V_base              # [A] base current
Z_base = V_base**2/S_base           # [Ω] base impedance
Y_base = 1/Z_base                   # [S] base admittance
print('====== System ======')
print('S_base = {:7.3f} MVA'.format(S_base*1e-6))
print('V_base = {:7.3f} kV'.format(V_base*1e-3))
print('I_base = {:7.3f} kA'.format(I_base*1e-3))
print('Z_base = {:7.3f} Ω'.format(Z_base))
print('Y_base = {:7.3f} S'.format(Y_base))

====== System ======
S_base =   1.000 MVA
V_base =  10.000 kV
I_base =   0.100 kA
Z_base = 100.000 Ω
Y_base =   0.010 S


#### Generator parameters
Base parameters of the single generator in the network

In [6]:
S_base_gen = S['G1']*1e6                      # [VA]
V_base_gen = PF_gen['Vl']*1e3 / PF_gen['u']   # [V]
I_base_gen = S_base_gen/V_base_gen            # [A]
Z_base_gen = V_base_gen**2 / S_base_gen       # [Ω]
Y_base_gen = 1 / Z_base_gen                   # [S]
rstr,xstr = 0.2, 0.4                          # [pu] stator parameters
R_gen,X_gen = rstr*Z_base_gen,xstr*Z_base_gen # [Ω]
Z_gen = R_gen + 1j*X_gen                      # [Ω]
Y_gen = 1/Z_gen                               # [S]
Z_gen_pu = Z_gen/Z_base                       # [pu]
Y_gen_pu = Y_gen/Y_base                       # [pu]
R_gen_pu,X_gen_pu = Z_gen_pu.real, Z_gen_pu.imag
print('===== Generator ====')
print('S_base = {:7.3f} MVA'.format(S_base_gen*1e-6))
print('V_base = {:7.3f} kV'.format(V_base_gen*1e-3))
print('I_base = {:7.3f} kA'.format(I_base_gen*1e-3))
print('Z_base = {:7.3f} Ω'.format(Z_base_gen))
print('Y_base = {:7.3f} S'.format(Y_base_gen))

===== Generator ====
S_base =  50.000 MVA
V_base =  10.000 kV
I_base =   5.000 kA
Z_base =   2.000 Ω
Y_base =   0.500 S


#### Load parameters
The p.u. values of the load impedance are referred to the system base.

In [7]:
load_type = 'const_S'
load_coeffs = np.zeros((2,2))
if load_type == 'const_Z':
    G_load = PF_load['P']*1e6 / (PF_bus['Vl']*1e3)**2  # [S]
    B_load = -PF_load['Q']*1e6 / (PF_bus['Vl']*1e3)**2 # [S]
    Y_load = G_load + 1j*B_load # [S]
    Y_load_pu = Y_load/Y_base   # [pu]
    Z_load = 1/Y_load           # [Ω]
    Z_load_pu = Z_load/Z_base   # [pu]
    G_load_pu,B_load_pu = Y_load_pu.real, Y_load_pu.imag
    load_coeffs = np.array([[G_load_pu,-B_load_pu],[B_load_pu,G_load_pu]])
    print('========== Load =========')
    print('G = {:7.5f} S, {:5.3f} pu'.format(G_load, G_load_pu))
    print('B = {:7.5f} S, {:5.3f} pu'.format(B_load, B_load_pu))
elif load_type == 'const_S':
    P = PF_load['P']*1e6/S_base  # [pu]
    Q = PF_load['Q']*1e6/S_base  # [pu]
    ur,ui = PF_bus['ur'], PF_bus['ui']
    den = (ur**2+ui**2)**2       # [pu]
    load_coeffs = np.array([[(P*(ui**2-ur**2) - 2*Q*ur*ui) / den, (Q*(ur**2-ui**2) - 2*P*ur*ui) / den],
                            [(Q*(ur**2-ui**2) - 2*P*ur*ui) / den, (P*(ur**2-ui**2) + 2*Q*ur*ui) / den]])
    print('======== Load ========')
    print('S = {:g} MVA = {:g} pu'.format(PF_load['P']+1j*PF_load['Q'], P+1j*Q))
else:
    raise Exception('Unknown load type `{load_type}`')

======== Load ========
S = 5-1j MVA = 5-1j pu


## Power flow results
#### Generator

In [8]:
print('     P = {:7.3f} MW'.format(PF_gen['P']))
print('     Q = {:7.3f} Mvar'.format(PF_gen['Q']))
print('     u = {:7.3f} pu'.format(PF_gen['u']))
print('    ur = {:7.3f} pu'.format(PF_gen['ur']))
print('    ui = {:7.3f} pu'.format(PF_gen['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_gen['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_gen['V']))
print('   V_ϕ = {:7.3f} deg'.format(PF_gen['phiu']))
print('     I = {:7.3f} kA'.format(PF_gen['I']))
print('     i = {:7.3f} pu'.format(PF_gen['i']))
print('    ir = {:7.3f} pu'.format(PF_gen['ir']))
print('    ii = {:7.3f} pu'.format(PF_gen['ii']))

     P =   5.000 MW
     Q =  -1.000 Mvar
     u =   0.900 pu
    ur =   0.900 pu
    ui =   0.000 pu
 V_l2l =   9.000 kV
 V_l2g =   5.196 kV
   V_ϕ =   0.000 deg
     I =   0.327 kA
     i =   0.113 pu
    ir =   0.111 pu
    ii =   0.022 pu


#### Load

In [9]:
print('     P = {:7.3f} MW'.format(PF_load['P']))
print('     Q = {:7.3f} Mvar'.format(PF_load['Q']))
print('     u = {:7.3f} pu'.format(PF_load['u']))
print('    ur = {:7.3f} pu'.format(PF_load['ur']))
print('    ui = {:7.3f} pu'.format(PF_load['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_load['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_load['V']))
print('   V_ϕ = {:7.3f} deg'.format(PF_load['phiu']))
print('     I = {:7.3f} kA'.format(PF_load['I']))
print('     i = {:7.3f} pu'.format(PF_load['i']))
print('    ir = {:7.3f} pu'.format(PF_load['ir']))
print('    ii = {:7.3f} pu'.format(PF_load['ii']))

     P =   5.000 MW
     Q =  -1.000 Mvar
     u =   0.900 pu
    ur =   0.900 pu
    ui =   0.000 pu
 V_l2l =   9.000 kV
 V_l2g =   5.196 kV
   V_ϕ =   0.000 deg
     I =   0.327 kA
     i =   0.113 pu
    ir =   0.111 pu
    ii =   0.022 pu


#### Bus

In [10]:
print('     u = {:7.3f} pu'.format(PF_bus['u']))
print('    ur = {:7.3f} pu'.format(PF_bus['ur']))
print('    ui = {:7.3f} pu'.format(PF_bus['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_bus['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_bus['V']))
print('     ϕ = {:7.3f} deg'.format(PF_bus['phi']))

     u =   0.900 pu
    ur =   0.900 pu
    ui =   0.000 pu
 V_l2l =   9.000 kV
 V_l2g =   5.196 kV
     ϕ =   0.000 deg


### Static load
  1. A static load is represented as a constant impedance.
  1. The number of variables is equal to 6.
  1. The submatrix has the following structure, where all the values are in per unit:

||$\frac{\partial}{\partial\phi}$|$\frac{\partial}{\partial u_r}$|$\frac{\partial}{\partial u_i}$|$\frac{\partial}{\partial i_r^G}$|$\frac{\partial}{\partial i_i^G}$|
|:---:|:---:|:---:|:---:|:---:|:---:|
|**$u_r$**| 0 | $G_L$ | $-B_L$ | -1 |  0 |
|**$u_i$**| 0 | $B_L$ | $G_L$ |  0 | -1 |
|**$i_r^G$**| $E_0\sin(\phi)$ | 1 | 0 | $R_G$ | $-X_G$ |
|**$i_i^G$**| $E_0\cos(\phi)$ | 0 | 1 | $X_G$ | $R_G$ |

where $(u_r,u_i)$ are the real and imaginary parts of the voltage at the bus, $(i_r^G,i_i^G)$ are the real and imaginary parts of the generator current, $(R_G,X_G)$ are the resistance and reactance of the synchronous machine's stator, and $(G_L,B_L)$ are the conductance and susceptance of the load.

### Dynamic load
  1. Dynamic load with constant power.
  1. The number of variables is equal to 8.
  1. The submatrix has the following structure, where all the values are in per unit:

|Variable|$\frac{\partial}{\partial\phi}$|$\frac{\partial}{\partial u_r}$|$\frac{\partial}{\partial u_i}$|$\frac{\partial}{\partial i_r^G}$|$\frac{\partial}{\partial i_i^G}$|$\frac{\partial}{\partial i_r^L}$|$\frac{\partial}{\partial i_i^L}$|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**$u_r$**| 0 | 0 | 0 | 1 | 0 | -1 |  0 |
|**$u_i$**| 0 | 0 | 0 | 0 | 1 |  0 | -1 |
|**$i_r^G$**| 0 | $c_{11}$ | $c_{12}$ | -1 |  0 | 0 | 0 |
|**$i_i^G$**| 0 | $c_{21}$ | $c_{22}$ |  0 | -1 | 0 | 0 |
|**$i_r^L$**| $E_0 \sin(\phi)$ | 1 | 0 | 0 | 0 | $R_G$ | $-X_G$ |
|**$i_i^L$**| $-E_0 \cos(\phi)$ | 0 | 1 | 0 | 0 | $X_g$ | $R_G$ |

where $(u_r,u_i)$ are the real and imaginary parts of the voltage at the bus, $(i_r^G,i_i^G)$ are the real and imaginary parts of the generator current, $(i_r^L,i_i^L)$ are the real and imaginary parts of the load current, and $(R_G,X_G)$ are the resistance and reactance of the synchronous machine's stator.

The coefficients $c_{11}$, $c_{12}$, $c_{21}$ and $c_{22}$ are given by the following expressions:

$c_{11} = \frac{\partial}{\partial u_r} \frac{P u_r + Q u_i}{u_r^2 + u_i^2} = \frac{P(u_i^2-u_r^2) - 2Q u_r u_i}{(u_r^2+u_i^2)^2}$

$c_{12} = \frac{\partial}{\partial u_i} \frac{P u_r + Q u_i}{u_r^2 + u_i^2} = \frac{Q(u_r^2-u_i^2) - 2P u_r u_i}{(u_r^2+u_i^2)^2}$

$c_{21} = \frac{\partial}{\partial u_r} \frac{-Q u_r + P u_i}{u_r^2 + u_i^2} = \frac{Q(u_r^2-u_i^2) - 2P u_r u_i}{(u_r^2+u_i^2)^2}$

$c_{22} = \frac{\partial}{\partial u_i} \frac{-Q u_r + P u_i}{u_r^2 + u_i^2} = \frac{P(u_r^2-u_i^2) + 2Q u_r u_i}{(u_r^2+u_i^2)^2}$

In [11]:
V = (Z_gen/Z_base_gen) * (PF_gen['ir'] + 1j*PF_gen['ii'])
e = (PF_bus['ur'] + 1j*PF_bus['ui']) + V
E0,ϕg = np.abs(e), np.angle(e)

In [12]:
IDX = lambda name: var_names.index(name)
J_guess = np.zeros((n_vars,n_vars), dtype=float)

In [13]:
if n_vars == 6:
    idx = [IDX('Bus1.ur'), IDX('Bus1.ui'), IDX('G1.ir:bus1'), IDX('G1.ii:bus1')]
    jdx = [IDX('G1.phi'), IDX('Bus1.ur'), IDX('Bus1.ui'), IDX('G1.ir:bus1'), IDX('G1.ii:bus1')]

    J_guess[idx[0],jdx[0]] =  0.
    J_guess[idx[0],jdx[1]] =  load_coeffs[0,0]
    J_guess[idx[0],jdx[2]] =  load_coeffs[0,1]
    J_guess[idx[0],jdx[3]] = -1.
    J_guess[idx[0],jdx[4]] =  0.

    J_guess[idx[1],jdx[0]] =  0.
    J_guess[idx[1],jdx[1]] =  load_coeffs[1,0]
    J_guess[idx[1],jdx[2]] =  load_coeffs[1,1]
    J_guess[idx[1],jdx[3]] =  0.
    J_guess[idx[1],jdx[4]] = -1.

    J_guess[idx[2],jdx[0]] =  E0*np.sin(ϕg)
    J_guess[idx[2],jdx[1]] =  1.
    J_guess[idx[2],jdx[2]] =  0.
    J_guess[idx[2],jdx[3]] =  R_gen_pu
    J_guess[idx[2],jdx[4]] = -X_gen_pu

    J_guess[idx[3],jdx[0]] = -E0*np.cos(ϕg)
    J_guess[idx[3],jdx[1]] =  0.
    J_guess[idx[3],jdx[2]] =  1.
    J_guess[idx[3],jdx[3]] =  X_gen_pu
    J_guess[idx[3],jdx[4]] =  R_gen_pu

In [14]:
if n_vars == 8:
    idx = [IDX('Bus1.ur'), IDX('Bus1.ui'), IDX('G1.ir:bus1'), IDX('G1.ii:bus1'),
           IDX('LD1.ir:bus1'), IDX('LD1.ii:bus1')]
    jdx = [IDX('G1.phi'), IDX('Bus1.ur'), IDX('Bus1.ui'), IDX('G1.ir:bus1'), IDX('G1.ii:bus1'),
           IDX('LD1.ir:bus1'), IDX('LD1.ii:bus1')]
    
    J_guess[idx[0],jdx[0]] =  0.
    J_guess[idx[0],jdx[1]] =  0.
    J_guess[idx[0],jdx[2]] =  0.
    J_guess[idx[0],jdx[3]] =  1.
    J_guess[idx[0],jdx[4]] =  0.
    J_guess[idx[0],jdx[5]] = -1.
    J_guess[idx[0],jdx[6]] =  0.

    J_guess[idx[1],jdx[0]] =  0.
    J_guess[idx[1],jdx[1]] =  0.
    J_guess[idx[1],jdx[2]] =  0.
    J_guess[idx[1],jdx[3]] =  0.
    J_guess[idx[1],jdx[4]] =  1.
    J_guess[idx[1],jdx[5]] =  0.
    J_guess[idx[1],jdx[6]] = -1.

    J_guess[idx[2],jdx[0]] =  0.
    J_guess[idx[2],jdx[1]] =  load_coeffs[0,0]
    J_guess[idx[2],jdx[2]] =  load_coeffs[0,1]
    J_guess[idx[2],jdx[3]] = -1.
    J_guess[idx[2],jdx[4]] =  0.
    J_guess[idx[2],jdx[5]] =  0.
    J_guess[idx[2],jdx[6]] =  0.

    J_guess[idx[3],jdx[0]] =  0.
    J_guess[idx[3],jdx[1]] =  load_coeffs[1,0]
    J_guess[idx[3],jdx[2]] =  load_coeffs[1,1]
    J_guess[idx[3],jdx[3]] =  0.
    J_guess[idx[3],jdx[4]] = -1.
    J_guess[idx[3],jdx[5]] =  0.
    J_guess[idx[3],jdx[6]] =  0.

    J_guess[idx[4],jdx[0]] =  E0*np.sin(ϕg)
    J_guess[idx[4],jdx[1]] =  1.
    J_guess[idx[4],jdx[2]] =  0.
    J_guess[idx[4],jdx[3]] =  0.
    J_guess[idx[4],jdx[4]] =  0.
    J_guess[idx[4],jdx[5]] =  R_gen_pu
    J_guess[idx[4],jdx[6]] = -X_gen_pu

    J_guess[idx[5],jdx[0]] = -E0*np.cos(ϕg)
    J_guess[idx[5],jdx[1]] =  0.
    J_guess[idx[5],jdx[2]] =  1.
    J_guess[idx[5],jdx[3]] =  0.
    J_guess[idx[5],jdx[4]] =  0.
    J_guess[idx[5],jdx[5]] =  X_gen_pu
    J_guess[idx[5],jdx[6]] =  R_gen_pu

In [15]:
if n_vars not in (6,8):
    raise Exception(f'Do not know how to deal with {n_vars} variables')

In [16]:
print_matrix(J[:n_vars,:n_vars], var_names, 'J.out' if n_vars > 8 else None)

          G1.speed        G1.phi       Bus1.ur       Bus1.ui    G1.ir:bus1    G1.ii:bus1   LD1.ir:bus1   LD1.ii:bus1 
[01]     -0.012821             0    -0.0138889   -0.00277778             0             0   -0.00236111   -2.22222e-05 
[02]             0             0             0             0             0             0             0             0 
[03]             0             0       0.00011             0             1             0            -1             0 
[04]             0             0             0       0.00011             0             1             0            -1 
[05]             0             0      -6.17285      -1.23457            -1             0             0             0 
[06]             0             0      -1.23457       6.17283             0            -1             0             0 
[07]             0     0.0488889             1             0             0             0         0.004        -0.008 
[08]             0     -0.913333             0         

In [17]:
print_matrix(J_guess, var_names, 'J_guess.out' if n_vars > 8 else None)

          G1.speed        G1.phi       Bus1.ur       Bus1.ui    G1.ir:bus1    G1.ii:bus1   LD1.ir:bus1   LD1.ii:bus1 
[01]             0             0             0             0             0             0             0             0 
[02]             0             0             0             0             0             0             0             0 
[03]             0             0             0             0             1             0            -1             0 
[04]             0             0             0             0             0             1             0            -1 
[05]             0             0      -6.17284      -1.23457            -1             0             0             0 
[06]             0             0      -1.23457       6.17284             0            -1             0             0 
[07]             0     0.0488889             1             0             0             0         0.004        -0.008 
[08]             0     -0.913333             0          